# Application of simple neural network in forest inventory #

#### This a notebook exercise is about the usage of a fully connected shallow neural network in estimating forest attributes.

In [ ]:
library(keras)

In [ ]:
source("../functions/keras_tf_funcs.R")

In [ ]:
# loading plot information
sp.data.train <- read.csv("../data/AV.leaf.on.train.csv",as.is=T)
sp.data.val <- read.csv("../data/AV.leaf.on.val.csv",as.is=T)
sp.data.test <- read.csv("../data/AV.leaf.on.test.csv",as.is=T)

# loading features
feat <- readRDS("../data/las.feat.AV-MK.leaf.on.RDS")

feat.train <- feat[feat$sampleplotid%in%sp.data.train$sampleplotid,]
feat.val <- feat[feat$sampleplotid%in%sp.data.val$sampleplotid,]
feat.test <- feat[feat$sampleplotid%in%sp.data.test$sampleplotid,]

# dropping columns w/o useful information (standard deviation is 0 or NaN/NA)
feat.train <- feat.train[,apply(feat.train,2,function(x) !(sd(x)==0|is.na(sd(x))))]
feat.val <- feat.val[,apply(feat.val,2,function(x) !(sd(x)==0|is.na(sd(x))))]
feat.test <- feat.test[,apply(feat.test,2,function(x) !(sd(x)==0|is.na(sd(x))))]
# keeping only features common in the two sets
feat.common <- Reduce(intersect,list(names(feat.train),names(feat.val),names(feat.test)))
feat.train <- feat.train[,feat.common]
feat.val <- feat.val[,feat.common]
feat.test <- feat.test[,feat.common]; rm(feat.common)

# adjusting column names
names(sp.data.train) <- gsub("\\.","_",names(sp.data.train))
names(sp.data.val) <- gsub("\\.","_",names(sp.data.val))
names(sp.data.test) <- gsub("\\.","_",names(sp.data.test))

# plot(sp.data$h,feat$hq85.f)
cor(sp.data.train$h,feat.train$hq85.f) #OK
cor(sp.data.val$h,feat.val$hq85.f) #OK
cor(sp.data.test$h,feat.test$hq85.f) #OK

# forest attributes
for.attrs <- c("v","h","d")

# datasets
d.sets <- c("train","val","test")

# creating input for TF
if (grepl("id|sampleplotid",names(feat.train)[1])) train.data <- feat.train[,-1] else train.data <- feat.train
if (grepl("id|sampleplotid",names(feat.val)[1])) val.data <- feat.val[,-1] else val.data <- feat.val
if (grepl("id|sampleplotid",names(feat.test)[1])) test.data <- feat.test[,-1] else test.data <- feat.test
train.labels <- sp.data.train[,for.attrs]
val.labels <- sp.data.val[,for.attrs]
test.labels <- sp.data.test[,for.attrs]


In [ ]:
n.neur <- 6

# declaring functions used during training
opt <- "adam"; act <- "swish"
# output directory
out.dir <- paste0("../AV_3layers_",n.neur,"neur_",opt,"_",act,"_multi")

preds.w <- c(0.6,0.2,0.2)
tf.runs <- lapply(1:3,function(i) {
  IRdisplay::display(paste0("Running iteration #",i))
  out.tmp <- tf.model.res(train.data,train.labels,val.data,val.labels,n.neur=n.neur,
                          scl="both",preds.w=preds.w,act=act,opt=opt,epochs=200,
                          patience=20,batch_size=25,metric="rmse",outdir=out.dir)
  row.names(out.tmp) <- paste0(row.names(out.tmp),".run",i)
  return(out.tmp)
}); cat("\n")